In [ ]:
import pandas as pd
import pandas as pd

from sklearn.model_selection import train_test_split

from kmodes.kmodes import KModes
from sklearn.model_selection import train_test_split
from pycaret.classification import *

%run class/patient_details.py
%run util/utils.py


Cardiovascular Disease Predictor

In [ ]:
# Cardiovascular
cardio_df = pd.read_csv("data/cardio_clean.csv")

df_male = cardio_df.query("gender == 1")
df_female = cardio_df.query("gender == 0")

# female data
# Building the model with using K-Mode with "Huang" initialization
km_huang = KModes(n_clusters=2, init="Huang", n_init=5, verbose=0)

male_kmode = km_huang.fit(df_male)
female_kmode = km_huang.fit(df_female)

clusters_female = km_huang.fit_predict(df_female)
clusters_male = km_huang.fit_predict(df_male)

# female DataFrame with Clusters
df_female.insert(0, "cluster", clusters_female, True)

# male DataFrame with Clusters
df_male.insert(0, "cluster", clusters_male, True)

# replacing cluster column values to merge dataframes after
df_male["cluster"].replace({0: 2, 1: 3}, inplace=True)

# merging female and male data
df_clusters = pd.concat([df_female, df_male], ignore_index=True, sort=False)

clf1 = setup(data=df_clusters,
             target='cardio',
             preprocess=False,
             silent=True)

models()

top5_cvd = compare_models(sort='AUC',
                          n_select=5,
                          )
blend_soft_cvd = blend_models(
    estimator_list=top5_cvd, optimize='AUC', method='soft')

cali_model_cvd = calibrate_model(blend_soft_cvd)

final_model_cvd = finalize_model(cali_model_cvd)


Diabetes Predictor

In [ ]:
# Diabetes
diabetes_df = pd.read_csv('data/diabetes_clean.csv')
diabetes_df.info()

X = diabetes_df.iloc[:, :-1]
y = diabetes_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42)

clf2 = setup(data=diabetes_df,
             target='Outcome',
             preprocess=False,
             silent=True)

models()

top5_diabetes = compare_models(sort='AUC', n_select=5)

blend_soft_diabetes = blend_models(
    estimator_list=top5_diabetes, optimize='AUC', method='soft')

cali_model_diabetes = calibrate_model(blend_soft_diabetes)

final_model_diabetes = finalize_model(cali_model_diabetes)


Reusable Functions

In [ ]:
def predict_diabetes(patient_details):
    prediction_diabetes = final_model_diabetes.predict([[
        patient_details.pregnancies,
        patient_details.glucose,
        patient_details.diastolic_bp,
        patient_details.skin_thickness,
        patient_details.insulin,
        patient_details.bmi,
        patient_details.dpf,
        patient_details.age
    ]])
    return prediction_diabetes[0]

def predict_cvd(patient_details):
    prediction_cvd = final_model_cvd.predict([[
        patient_details.cluster,
        patient_details.gender,
        patient_details.age_class,
        patient_details.bmi_class,
        patient_details.map_class,
        patient_details.cholesterol_class,
        patient_details.glucose_class,
        patient_details.smoke,
        patient_details.active
    ]])
    return prediction_cvd[0]

def assign_cluster_male(patient):
    cluster = male_kmode.predict(patient)
    if cluster[0] == 0:
        return 2
    if cluster[0] == 1:
        return 3

def assign_cluster_female(patient):
    predict = female_kmode.predict(patient)
    return predict[0]

def assign_cluster(patient_details):
    if patient_details.gender == 0:  # Female
        return assign_cluster_female([[
            patient_details.gender,
            patient_details.age_class,
            patient_details.bmi_class,
            patient_details.map_class,
            patient_details.cholesterol_class,
            patient_details.glucose_class,
            patient_details.smoke,
            patient_details.active,
            patient_details.cvd_prediction
        ]])
    else:
        return assign_cluster_male([[
            patient_details.gender,
            patient_details.age_class,
            patient_details.bmi_class,
            patient_details.map_class,
            patient_details.cholesterol_class,
            patient_details.glucose_class,
            patient_details.smoke,
            patient_details.active,
            patient_details.cvd_prediction
        ]])

def proccess_patient_details(age, gender, pregnancies, glucose, cholesterol, insulin, systolic_bp, diastolic_bp, skin_thickness, bmi, smoke, active, dpf):
    patient_details = Patient()

    patient_details.age = age
    patient_details.gender = gender
    patient_details.pregnancies = pregnancies
    patient_details.glucose = glucose
    patient_details.cholesterol = cholesterol
    patient_details.insulin = insulin
    patient_details.systolic_bp = systolic_bp
    patient_details.diastolic_bp = diastolic_bp
    patient_details.skin_thickness = skin_thickness
    patient_details.bmi = bmi
    patient_details.smoke = smoke
    patient_details.active = active
    patient_details.dpf = dpf

    #Initial value, might change after prediction
    patient_details.cvd_prediction = 0

    patient_details = process_patient_classification(patient_details)

    return patient_details

def process_patient_classification(patient_details):
    patient_details.mean_arterial_pressure = compute_mean_arterial_pressure(patient_details.systolic_bp, patient_details.diastolic_bp)
    patient_details.age_class = get_age_class(patient_details.age)
    patient_details.bmi_class = get_bmi_class(patient_details.bmi)
    patient_details.map_class = get_map_class(patient_details.mean_arterial_pressure)
    patient_details.cholesterol_class = get_cholesterol_class(patient_details.cholesterol)
    patient_details.glucose_class = get_glucose_class(patient_details.glucose)

    patient_details.cluster = assign_cluster(patient_details)

    patient_details.cvd_prediction = predict_cvd(patient_details)
    patient_details.diabetes_prediction = predict_diabetes(patient_details)

    return patient_details



In [ ]:
def create_result_output(patient_map):
    output = html.Div([
        html.H1('Patient Health Status: ', className='result-title display-4 mb-3'),
        html.P(children=[
            html.B("Body Mass Index: "),
            patient_map["health_statuses"]["bmi_status"]
        ]),        
        html.P(children=[
            html.B("Mean Arterial Pressure: "),
            patient_map["health_statuses"]["map_status"]
        ], className="mb-3"),        
        html.P(children=[
            html.B("Glucose level: "),
            patient_map["health_statuses"]["glucose_status"]
        ], className="mb-3"),        
        html.P(children=[
            html.B("Cholesterol level: "),
            patient_map["health_statuses"]["cholesterol_status"]
        ], className="mb-3"),             
        html.P(children=[
            html.B("Rish of Cardiovascular Disease: "),
            patient_map["health_statuses"]["cvd_risk"]
        ], className="mb-3"),
        html.P(children=[
            html.B("Risk of Diabetes: "),
            patient_map["health_statuses"]["diabetes_risk"]
        ], className="mb-3"),
    ])
    return output

Web Application Implementation

In [14]:
import json
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    html.Div([
        html.H1(children='Halo Web Prototype',
                className='page-title display-4'),
        html.Hr(className='w-25'),
        html.P(children='This application predicts the risk of a patient having cardiovascular disease and diabetes.', className="lead")
        
    ], className="w-100"),
    dbc.Row([
        dbc.Col([
            dbc.Card([

                html.Div(
                    id="feedback-container",
                    children=[
                    ]),

                html.Div([
                    dbc.Label("Age"),
                    dbc.Input(id="age", type="number",  min=0,
                              placeholder="Enter age"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Gender"),
                    dcc.Dropdown(
                        id="gender",
                        options=[
                            {'label': 'Male', 'value': 1, },
                            {'label': 'Female', 'value': 0}
                        ],
                        placeholder="Please select"
                    ),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Number of Pregnancies"),
                    dbc.Input(id="pregnancies", type="number",  min=0,
                              placeholder="Enter number of pregnancies"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Glucose Level"),
                    dbc.Input(id="glucose", type="number",  min=0,
                              placeholder="Enter glucose level"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Cholesterol Level"),
                    dbc.Input(id="cholesterol", type="number",  min=0,
                              placeholder="Enter cholesterol level"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Insulin Level"),
                    dbc.Input(id="insulin", type="number",  min=0,
                              placeholder="Enter insulin level"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Systolic Blood Pressure"),
                    dbc.Input(id="systolic_bp", type="number",  min=0,
                              placeholder="Enter blood pressure here"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Diastolic Blood Pressure"),
                    dbc.Input(id="diastolic_bp", type="number", min=0,
                              placeholder="Enter blood pressure here"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Skin Thickness"),
                    dbc.Input(id="skin_thickness", type="number",
                              min=0, placeholder="Enter thickness"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Body Mass Index (BMI)"),
                    dbc.Input(id="bmi", type="number",  min=0,
                              placeholder="Enter BMI"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Do You Smoke?"),
                    dcc.Dropdown(
                        id="smoke",
                        options=[
                            {'label': 'Yes', 'value': 1},
                            {'label': 'No', 'value': 0}
                        ],
                        placeholder="Please select"
                    ),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Are You Physically Active?"),
                    dcc.Dropdown(
                        id="active",
                        options=[
                            {'label': 'Yes', 'value': 1},
                            {'label': 'No', 'value': 0}
                        ],
                        placeholder="Please select"
                    ),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Does your family have a history of diabetes?"),
                    dcc.Dropdown(
                        id="dpf",
                        options=[
                            {'label': 'Yes', 'value': 1},
                            {'label': 'No', 'value': 0}
                        ],
                        placeholder="Please select"
                    ),
                ], className='form-container mb-3'),
            ], body=True,
                style={
                'margin-top': '0.5rem',
                'margin-bottom': '1rem',
                }
            ),

            html.Div([
                dbc.Button('Predict', id='submit-val',
                           color='primary', className='w-100 mb-3'),
            ]),

        ]),

        dbc.Col([
            dbc.Card([
                html.H1("Prediction Results", className='display-4'),
                html.Div(
                    id="output-container",
                    children=[
                    ]),
            ], body=True, 
                style={
                'margin-top': '0.5rem',
                'margin-bottom': '1rem', 
                }
            ),
        ]),
    ]),
],
    fluid=True,
)

@app.callback(
    Output(component_id='feedback-container', component_property='children'),
    Input(component_id='submit-val', component_property='n_clicks'),
    Input(component_id='age', component_property='value'),
    Input(component_id='gender', component_property='value'),
    Input(component_id='glucose', component_property='value'),
    Input(component_id='cholesterol', component_property='value'),
    Input(component_id='systolic_bp', component_property='value'),
    Input(component_id='diastolic_bp', component_property='value'),
    Input(component_id='pregnancies', component_property='value'),
    Input(component_id='skin_thickness', component_property='value'),
    Input(component_id='insulin', component_property='value'),
    Input(component_id='bmi', component_property='value'),
    Input(component_id='dpf', component_property='value'),
    Input(component_id='smoke', component_property='value'),
    Input(component_id='active', component_property='value')
)

def send_feedback(n_clicks, age, gender, glucose, pregnancies, cholesterol, systolic_bp, diastolic_bp, skin_thickness, insulin, bmi, dpf, smoke, active):
    if (n_clicks is not None and
        (age is None
        or gender is None
        or pregnancies is None
        or glucose is None
        or cholesterol is None
        or systolic_bp is None
        or diastolic_bp is None
        or skin_thickness is None
        or insulin is None
        or bmi is None
        or dpf is None
        or smoke is None
        or active is None)):
        return dbc.Alert('All fields are required', color='danger')

@app.callback(
    Output(component_id='output-container', component_property='children'),
    Input(component_id='submit-val', component_property='n_clicks'),
    Input(component_id='age', component_property='value'),
    Input(component_id='gender', component_property='value'),
    Input(component_id='pregnancies', component_property='value'),
    Input(component_id='glucose', component_property='value'),
    Input(component_id='cholesterol', component_property='value'),
    Input(component_id='insulin', component_property='value'),
    Input(component_id='systolic_bp', component_property='value'),
    Input(component_id='diastolic_bp', component_property='value'),
    Input(component_id='skin_thickness', component_property='value'),
    Input(component_id='bmi', component_property='value'),
    Input(component_id='smoke', component_property='value'),
    Input(component_id='active', component_property='value'),
    Input(component_id='dpf', component_property='value')
)

def update_output(n_clicks, age, gender, pregnancies, glucose, cholesterol,  insulin, systolic_bp, diastolic_bp, skin_thickness, bmi, smoke, active, dpf):
    if (n_clicks is not None
        and age is not None
        and gender is not None
        and pregnancies is not None
        and glucose is not None
        and cholesterol is not None
        and insulin is not None
        and systolic_bp is not None
        and diastolic_bp is not None
        and skin_thickness is not None
        and bmi is not None
        and smoke is not None
        and active is not None
        and dpf is not None):
        
        patient = proccess_patient_details(age, gender, pregnancies, glucose, cholesterol, insulin, systolic_bp, diastolic_bp, skin_thickness, bmi, smoke, active, dpf)

        patient_map = {}

        patient_map["patient"] = patient.map_patient_details()
        
        patient_map["health_statuses"] = get_health_status(patient_map["patient"]["bmi_class"], patient_map["patient"]["map_class"], patient_map["patient"]["cholesterol_class"], patient_map["patient"]["glucose_class"], patient_map["patient"]["cvd_prediction"], patient_map["patient"]["diabetes_prediction"])

        patient_details_json = json.dumps(patient_map, indent=2)

        output = create_result_output(patient_map)
        print(patient_details_json)
        return output


if __name__ == '__main__':
    app.run_server(port=4050)
